# INSTALAR LIBRERIAS

In [13]:
# !pip install docx2pdf
#pip install ipywidgets

# IMPORTAR

In [1]:
from docx2pdf import convert
import os
import time

# CONSTRUIR LA RUTA DE LOS ARCHIVOS

In [2]:
def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../../../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [3]:
unidad = get_unidad_local()
ruta = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta)
ruta_completa

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [4]:
# colocar aqui la ruta de los archivos
path = ruta_completa + r'/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 2 INFORMES
2 INFORMES


['G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/2641_CRS_10-04-2025_CRONENBOLD AGUILERA ROBERTO_495.docx',
 'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/informes/40139_CRS_26-03-2025_ROCA SUAREZ MARIA ALEJANDRA_473.docx']

# INICIA EL PROCESO DE CONVERCIÓN

In [5]:
rutas[1].replace('.docx','.pdf').replace('informes','pdf')

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/pdf/40139_CRS_26-03-2025_ROCA SUAREZ MARIA ALEJANDRA_473.pdf'

In [6]:
rutas[1].replace('.docx','.pdf').replace('informes','pdf')

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON/pdf/40139_CRS_26-03-2025_ROCA SUAREZ MARIA ALEJANDRA_473.pdf'

In [8]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdfs').split('/')[-1]
    convert(filename, r'C:\Users\bismarksr\Downloads\\' + pdf)
    i+=1
    time.sleep(5)
print('END')

CONVIRTIENDO... INFORME 1 G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/2641_CRS_10-04-2025_CRONENBOLD AGUILERA ROBERTO_495.docx
G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/informes/2641_CRS_10-04-2025_CRONENBOLD AGUILERA ROBERTO_495.docx
G:\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON/pdfs/2641_CRS_10-04-2025_CRONENBOLD AGUILERA ROBERTO_495.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

com_error: (-2147352567, 'Ocurrió una excepción.', (0, 'Microsoft Word', 'El comando falló', 'wdmain11.chm', 36966, -2146824090), None)

# Leer archivos

In [10]:
# CONEXION A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1096764895697-57btfmtn7rdejufct5qq4j9mi61j0unl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [11]:
contenido = os.listdir(ruta_completa + r'/pdf')
dic_files = {}
for file in contenido:
    split = file.split('_')
    dic_files[int(split[0])] = file
print(len(dic_files), 'informes')
dic_files

2 informes


{2641: '2641_CRS_10-04-2025_CRONENBOLD AGUILERA ROBERTO_495.pdf',
 40139: '40139_CRS_26-03-2025_ROCA SUAREZ MARIA ALEJANDRA_473.pdf'}

# CARGAR LOS PDFS A GOOGLE DRIVE

In [12]:
# CARGAR LOS PDFS A GOOGLE DRIVE
dic_urls = {}
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  #print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "2DA_INSPECCION"):
      fileID = file['id']

for file in dic_files:
    file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    file1['title'] = dic_files[file]
    file1.SetContentFile(ruta_completa + r'/pdf//' + dic_files[file])
    file1.Upload() # Upload the file.
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))
    permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    dic_urls[file] = file1['alternateLink']
dic_urls
print('end')

Created file 2641_CRS_10-04-2025_CRONENBOLD AGUILERA ROBERTO_495.pdf with mimeType application/pdf
Created file 40139_CRS_26-03-2025_ROCA SUAREZ MARIA ALEJANDRA_473.pdf with mimeType application/pdf
end


In [13]:
print(len(dic_urls), 'URLs')
dic_urls

2 URLs


{2641: 'https://drive.google.com/file/d/1wI6ThNUVBp8LgAoshcLGvVhrqxqb_RXj/view?usp=drivesdk',
 40139: 'https://drive.google.com/file/d/1b9flLew6aqA95f0BXUIbSILTtm5XxlD5/view?usp=drivesdk'}

# conexion a amigocloud

In [14]:
import sys
sys.path.append('../../../_amigocloud')

from amigocloud import AmigoCloud

In [15]:
# leer token
acceso = open(ruta_completa + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [16]:
for key in dic_urls:
    query = {'query': 'update dataset_307560 set link_doc_2da = \'{link}\' where cast(SPLIT_PART(canhero, \' / \', 1) as integer) = \'{cod}\''.format(link=dic_urls[key], cod=key)}
    res = amigocloud.post('https://app.amigocloud.com/api/v1/projects/32672/sql', query)
    print(key)
print('end')

2641
40139
end
